<p style="text-align:center"><font size="10" > Exercise Answer 06</font></p>

---
---

# Exercise 06-1 解答(あくまでも田村の回答)

## 今回のテキストマイニングの部分

### テキストマイニング部分①
- メッセージデータから句読点や日本語以外の文字を削除してノイズを除去
- 前処理データの一部を用いたSVM分類用のトレーニングデータの作成
    - メッセージデータの中から1000個のメッセージをランダムに選択
    - ないため、Antweiler and Frank (2004)やDas and Chen (2007)に倣って手動で強気、弱気、中立のメッセージに分類
    - 次に、分類されたメッセージの中から、「売り」「買い」「高」など、強気・弱気の感情に大きく関係するキーワードを選択
    -  SVMを用いて、メッセージデータに含まれるキーワードの組み合わせと頻度に基づいて、メッセージを強気、弱気、中立に分類する
    
### テキストマイニング部分②
- 前処理されたすべてのメッセージはをMeCabを用いて解析しメッセージを言葉単位に分解
- 言葉単位に分解したデータを元に、学習させたSVMを使いすべてのメッセージを3つのセンチメントクラスに分類



## 課題や不確実な部分

### テキストマイニング部分①
1. 「第１段階、第２段階の各段階で収集したメッセージデータの中から1000個のメッセージをランダムに選択」とあるが、企業数分のデータを集めてランダムに選択するのは工数がかなりかかると予想。

2. 1000個のメッセージを手動で強気、弱気、中立のに分類（将来の企業業績や市場の状況について楽観的な意見を表すメッセージは強気、悲観的な意見を表すメッセージは弱気に分類。また、将来の企業業績や市況について明確な感情がないメッセージは、中立的なものに分類）は工数がかなりかかると予想。

3. 「フェーズ1では260個、フェーズ2では255個のキーワードを選択」の選択方法が選択者の主観に頼るため再現性が低いと予想。



### テキストマイニング部分②

4. 「情報やノイズの多い情報を除外するために167の単語を辞書に追加」とあるが詳細のデータを確認しないと再現性が低いと予想。

5. すべてのメッセージをキーワード頻度行列に変換　--> 実装方法が不明。

6. SVM利用し、3つのセンチメントクラスに分類　-> 5ができないと6を分類することが不可能。　

    
### 用語の確認
- フェーズ１・・最初のメッセージが掲載されてからブックビルディングが終了するまでの期間
- フェーズ２・・ブックビルディングが終了してからIPO直前までの期間
    
 ---

# Exercise 06-2 解答(あくまでも田村の回答)

## 実装の手順(TODO)


### テキストマイニング1の部分

1. 第１段階、第２段階の各段階で収集したメッセージデータの中から1000個のメッセージをランダムに選択
    1.  ランダムに企業コードを選択するプログラムを実行し、選択された企業名からcsvデータを読み込む
    2. 読み込まれたcsvデータのメッセージの項目から適当に1つ選択する
    3. 1, 2を1000回繰り返す
　
2. 1000個のメッセージを手動で強気、弱気、中立に分類

3. ２で、分類されたメッセージの中から、「売り」「買い」「高」など、強気・弱気の感情に大きく関係するキーワードを選択。


### テキストマイニング2の部分

4. macabなどの形態素解析ライブラリでメッセージの単語を分割する(その際に情報やノイズの多い情報を除外するために167の単語を辞書に追加)

5. 企業数分のメッセージ（形態素解析で分けられた単語群）配列をキーワード頻度行列に変換

6. SVM利用し、3つのセンチメントクラスに分類